In [12]:
import cv2
import torch
import transformers
from torchvision import models, transforms
from PIL import Image

In [3]:
#capturing from webcam
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if ret:
    cv2.imwrite('captured_image.jpg', frame)
cap.release()

In [16]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()


# Define a transform to convert image to tensor
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load the image and apply the transformation
image = Image.open('download.jpg')
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

In [6]:
#performing object detection
with torch.no_grad():
    predictions = model(image_tensor)

In [18]:
from pycocotools.coco import COCO

# Load COCO dataset metadata
coco = COCO("instances_val2017.json")  # Replace with the correct path to annotations

# Get the categories and their names
categories = coco.loadCats(coco.getCatIds())
coco_labels = {cat['id']: cat['name'] for cat in categories}

# Now, coco_labels will map from category ID to the corresponding name

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!


In [20]:
detected_objects = []
for element in range(len(predictions[0]['labels'])):
    label_id = predictions[0]['labels'][0].item()  # Example label ID from model output
    label = coco_labels[label_id]
    score = predictions[0]['scores'][element].item()
    if score > 0.5:  # Threshold to filter weak detections
        detected_objects.append(f"{label} (confidence: {score:.2f})")

description = "Detected objects: " + ", ".join(detected_objects)
print(description)

Detected objects: cup (confidence: 0.71)
